In [42]:
import keras
from keras.datasets import cifar10, mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, SGD
from keras import backend as K
from keras import regularizers


import numpy as np

In [6]:
# the data, shuffled and split between train and test sets
def load_data(dataset):
    # Load
    (x_train, y_train), (x_test, y_test) = dataset.load_data()
    # Reshape X
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    n_channels = x_train.shape[3] if len(x_train.shape) == 4 else 1
    if K.image_data_format() == 'channels_first':
        s = [n_channels, img_rows, img_cols]
    else:
        s = [img_rows, img_cols, n_channels]
    x_train = x_train.reshape(x_train.shape[0], *s)
    x_test = x_test.reshape(x_test.shape[0], *s)
    # Rescale X
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    # Reshape y
    num_classes = len(np.unique(y_train))
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    return (x_train, y_train), (x_test, y_test), img_rows, img_cols

In [7]:
(x_train, y_train), (x_test, y_test), img_rows, img_cols = load_data(mnist)

In [8]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [9]:
linear = Sequential()
linear.add(Flatten(input_shape=x_train.shape[1:]))
linear.add(Dense(units=10, activation='softmax'))

In [106]:
small_cnn = Sequential()

small_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=x_train.shape[1:]))
small_cnn.add(Conv2D(32, (3, 3), activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.25))

small_cnn.add(Conv2D(64, (3, 3), activation='relu'))
small_cnn.add(Conv2D(64, (3, 3), activation='relu'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.25))

small_cnn.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
small_cnn.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
small_cnn.add(MaxPooling2D(pool_size=(2, 2)))
small_cnn.add(Dropout(0.25))


small_cnn.add(Flatten())
small_cnn.add(Dense(64, activation='relu'))
small_cnn.add(Dropout(0.4))
small_cnn.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
small_cnn.add(Dense(units=10, activation='softmax'))

In [107]:
#model = linear
model = small_cnn

In [108]:
#optimizers: keras.optimizers.Adadelta()
#Adam(lr=0.001, decay=0., epsilon=1e-8)
#keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-8)
#opt = keras.optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)

In [109]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt,
              metrics=['accuracy'])

In [110]:
n_train    = 600
batch_size = 128
epochs     = 60

model.fit(x_train[:n_train], y_train[:n_train],
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test[:n_train], y_test[:n_train]))

Train on 600 samples, validate on 600 samples
Epoch 1/60
600/600 [==============================] - 3s - loss: 2.3220 - acc: 0.0867 - val_loss: 2.3009 - val_acc: 0.1217
Epoch 2/60
600/600 [==============================] - 2s - loss: 2.2915 - acc: 0.1567 - val_loss: 2.1784 - val_acc: 0.2067
Epoch 3/60
600/600 [==============================] - 2s - loss: 2.1729 - acc: 0.2317 - val_loss: 2.0330 - val_acc: 0.3383

In [77]:
_, acc = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', acc)

Test accuracy: 0.9504
